# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("Outputs/CSV/weather_ouput.csv", encoding = "utf-8")
weather_df

,city,country,lat,lng,date,temp,humidity,cloudiness,wind_speed
0,Pochutla,MX,15.74,-96.47,1583820944,77.00,67,0,12.75
1,Floresta,BR,-8.60,-38.57,1583820944,74.32,82,91,5.99
2,Arraial do Cabo,BR,-22.97,-42.03,1583820945,75.76,85,0,10.80
3,Jumla,NP,29.27,82.18,1583820945,60.01,92,0,5.46
4,Hong Kong,HK,22.29,114.16,1583820877,76.95,56,20,19.46
...,...,...,...,...,...,...,...,...,...
562,Severnoye,RU,56.35,78.36,1583821298,16.00,88,91,7.90
563,Valleyview,CA,55.07,-117.29,1583821298,19.60,80,0,6.49
564,Kununurra,AU,-15.77,128.73,1583821298,98.60,25,0,6.93
565,Tambun,ID,-6.26,107.07,1583821299,93.74,52,40,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#weather_df.rename(columns={"lat": "Lat", "lng": "Lng"}, inplace = True)
gmaps.configure(api_key = g_key)
locations = weather_df[["lat", "lng"]]
humidity = weather_df['humidity']
maxW = humidity.max()

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = maxW, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
narrow_df = weather_df.loc[(weather_df['temp'] > 70) & (weather_df['temp'] < 80)\
& (weather_df['wind_speed'] < 10) & (weather_df['cloudiness'] == 0)]

narrow_df.reset_index(inplace=True)
narrow_df

,index,city,country,lat,lng,date,temp,humidity,cloudiness,wind_speed
0,7,São João da Barra,BR,-21.64,-41.05,1583820945,75.22,84,0,2.66
1,12,Champerico,GT,14.30,-91.92,1583820946,76.12,76,0,4.21
2,18,Puerto Ayora,EC,-0.74,-90.35,1583820946,78.01,100,0,7.85
3,60,Pisco,PE,-13.70,-76.22,1583820953,72.18,83,0,5.82
4,96,Rāmpura,IN,30.25,75.23,1583820958,76.12,34,0,1.19
5,165,Pāsighāt,IN,28.07,95.33,1583821188,75.04,52,0,4.14
6,172,Puerto Escondido,MX,15.85,-97.07,1583821189,71.80,65,0,3.98
7,177,Port Lincoln,AU,-34.73,135.87,1583821190,74.68,44,0,8.95
8,272,Mount Gambier,AU,-37.83,140.77,1583821201,77.00,27,0,8.05
9,380,Richards Bay,ZA,-28.78,32.04,1583821274,74.50,80,0,5.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel = []

for city in range(len(narrow_df)):
    lat = narrow_df.loc[city]['lat']
    lng = narrow_df.loc[city]['lng']
    format = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    query = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(query, params = format).json()
    try:
        hotel.append(response['results'][0]['name'])
    except:
        print("No Hotel Found")
        
narrow_df["hotel_name"] = hotel
narrow_df

C:\Users\tbuhr\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,city,country,lat,lng,date,temp,humidity,cloudiness,wind_speed,hotel_name
0,7,São João da Barra,BR,-21.64,-41.05,1583820945,75.22,84,0,2.66,Grussaí
1,12,Champerico,GT,14.30,-91.92,1583820946,76.12,76,0,4.21,Champerico
2,18,Puerto Ayora,EC,-0.74,-90.35,1583820946,78.01,100,0,7.85,Puerto Ayora
3,60,Pisco,PE,-13.70,-76.22,1583820953,72.18,83,0,5.82,Pisco
4,96,Rāmpura,IN,30.25,75.23,1583820958,76.12,34,0,1.19,Rampura Phul
5,165,Pāsighāt,IN,28.07,95.33,1583821188,75.04,52,0,4.14,Nyingchi
6,172,Puerto Escondido,MX,15.85,-97.07,1583821189,71.80,65,0,3.98,Brisas de Zicatela
7,177,Port Lincoln,AU,-34.73,135.87,1583821190,74.68,44,0,8.95,Port Lincoln
8,272,Mount Gambier,AU,-37.83,140.77,1583821201,77.00,27,0,8.05,Mount Gambier
9,380,Richards Bay,ZA,-28.78,32.04,1583821274,74.50,80,0,5.91,Richards Bay


In [6]:
#drop rows that didn't pull a hotel/match the name of the city
for city in range(len(narrow_df)):
    cit = narrow_df.loc[city]['city']
    hot = narrow_df.loc[city]['hotel_name']
    if cit == hot:
        narrow_df.drop(city, inplace = True)
narrow_df

C:\Users\tbuhr\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,index,city,country,lat,lng,date,temp,humidity,cloudiness,wind_speed,hotel_name
0,7,São João da Barra,BR,-21.64,-41.05,1583820945,75.22,84,0,2.66,Grussaí
4,96,Rāmpura,IN,30.25,75.23,1583820958,76.12,34,0,1.19,Rampura Phul
5,165,Pāsighāt,IN,28.07,95.33,1583821188,75.04,52,0,4.14,Nyingchi
6,172,Puerto Escondido,MX,15.85,-97.07,1583821189,71.80,65,0,3.98,Brisas de Zicatela


In [8]:
hotel_df = narrow_df
hotel_df

,index,city,country,lat,lng,date,temp,humidity,cloudiness,wind_speed,hotel_name
0,7,São João da Barra,BR,-21.64,-41.05,1583820945,75.22,84,0,2.66,Grussaí
4,96,Rāmpura,IN,30.25,75.23,1583820958,76.12,34,0,1.19,Rampura Phul
5,165,Pāsighāt,IN,28.07,95.33,1583821188,75.04,52,0,4.14,Nyingchi
6,172,Puerto Escondido,MX,15.85,-97.07,1583821189,71.80,65,0,3.98,Brisas de Zicatela


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["lat", "lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations2)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))